In [1]:
%load_ext autoreload
%autoreload 2

# 1. 病种高套（BZGY）基本建模思路
  
* 病种高套指在DRGs或者单病种付费支付制度下，医院高编病组,套取超额报销的违规行为。
  
<p>住院结束后医院需要把入院诊断，出院诊断和每日使用明细和花费上传给医保局. 医院可以轻易修改出院诊断，但每日明细耗材涉及到药品进销存等，不易篡改。因为我们可以使用大量住院单据，训练从明细->出院诊断的分类模型。对于每个住院单据，我们使用分类模型判断其出院诊断，如果模型输出和实际商保诊断不符，且模型推断的诊断对应报销金额大于医院实际上报诊断时，则该单据有较大的病种高套嫌疑</p>
  
  

 

  
  

# 2.**ETL**



## 2.1 环境变量

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

REGISTER_PATH = "./data/inHosp/register"
DETAIL_PATH = "./data/inHosp/detail"
COMBINED_PATH = "./data/inHosp/combined"

conf = SparkConf().setAppName('BZGT').setMaster('local[*]')

In [2]:
spark = SparkSession.builder.config(conf=conf).config("spark.driver.memory", "35g").config("spark.local.dir", "/media/data1/data/tmp").getOrCreate()
spark

In [3]:
#从医保报销的单据表中抽取住院单据以及其对应明细清单
register = spark.read.parquet(REGISTER_PATH)
detail = spark.read.parquet(DETAIL_PATH)
# combine = spark.read.parquet(COMBINED_PATH)
print(register.count())
print(detail.count())
# print(combine.count())

139229
44195071
136118


## 2.2 EDA

In [4]:
register.show()

+--------------------+----------+---+---+--------+----------+---------------------+--------------------------+--------------------------+----------------------+----------+-------------------+-------------------+------------+----------+----------+----------+-----------+-----------+--------+----------+---------+--------------+-------------------+
|         register_no|    per_no|age|sex|per_type| zone_name|in_hosp_diagnosiscode|     in_hosp_diagnosisname|    out_hosp_diagnosisname|out_hosp_diagnosiscode|visit_type|       in_hosp_date|      out_hosp_date|     hosp_no| hosp_name|hosp_level| dept_name|doctor_name|doctor_code|bill_sum|bill_apply|bill_self|rs_overall_pay|    settlement_date|
+--------------------+----------+---+---+--------+----------+---------------------+--------------------------+--------------------------+----------------------+----------+-------------------+-------------------+------------+----------+----------+----------+-----------+-----------+--------+----------+-----

In [3]:
register.columns

['register_no',
 'per_no',
 'age',
 'sex',
 'per_type',
 'zone_name',
 'in_hosp_diagnosiscode',
 'in_hosp_diagnosisname',
 'out_hosp_diagnosisname',
 'out_hosp_diagnosiscode',
 'visit_type',
 'in_hosp_date',
 'out_hosp_date',
 'hosp_no',
 'hosp_name',
 'hosp_level',
 'dept_name',
 'doctor_name',
 'doctor_code',
 'bill_sum',
 'bill_apply',
 'bill_self',
 'rs_overall_pay',
 'settlement_date']

In [7]:
detail.count()

['CHARGE_DETAIL_NO',
 'REGISTER_NO',
 'EXTERNAL_CD_STR_1',
 'MED_PROJECT_CODE',
 'MED_PROJECT_NAME',
 'PROJECT_TYPE',
 'HOSP_PROJECT_NAME',
 'SETTLEMENT_DATE',
 'RECIPE_DATE',
 'CHARGE_TYPE',
 'MEDICAL_SPECIFICATION',
 'DOSE_FORM',
 'PRICE',
 'QUANTITY',
 'MONEY',
 'CD_OVERALL_PAY',
 'APPLY_AMOUNT',
 'SELF_AMOUNT',
 'HOSP_PROJECT_CODE']

In [5]:
register.groupBy("out_hosp_diagnosisname").count().sort("count", ascending=False).show(10)

+------------------------+-----+
|  out_hosp_diagnosisname|count|
+------------------------+-----+
|                肺部感染| 3414|
|                    null| 2940|
|              支气管肺炎| 2934|
|                    肺炎| 2555|
|冠状动脉粥样硬化性心脏病| 2429|
|  社区获得性肺炎，非重症| 1938|
|                  脑梗死| 1687|
|                支气管炎| 1605|
|                结肠息肉| 1416|
|              肺恶性肿瘤| 1358|
+------------------------+-----+
only showing top 10 rows



In [19]:
detail.show(3)

+--------------------+--------------------+-----------------+----------------+----------------+------------+-----------------+-------------------+-------------------+-----------+---------------------+---------+-----+--------+-----+--------------+------------+-----------+-----------------+
|    CHARGE_DETAIL_NO|         REGISTER_NO|EXTERNAL_CD_STR_1|MED_PROJECT_CODE|MED_PROJECT_NAME|PROJECT_TYPE|HOSP_PROJECT_NAME|    SETTLEMENT_DATE|        RECIPE_DATE|CHARGE_TYPE|MEDICAL_SPECIFICATION|DOSE_FORM|PRICE|QUANTITY|MONEY|CD_OVERALL_PAY|APPLY_AMOUNT|SELF_AMOUNT|HOSP_PROJECT_CODE|
+--------------------+--------------------+-----------------+----------------+----------------+------------+-----------------+-------------------+-------------------+-----------+---------------------+---------+-----+--------+-----+--------------+------------+-----------+-----------------+
|220019279-1-60775...|220019279-1-33052...|        220042279|C01100201021056y|            甘草|      药品费|           炙甘草|2019-01-15 0

In [23]:
detail.groupBy("CHARGE_TYPE").count().show(100)

+---------------+--------+
|    CHARGE_TYPE|   count|
+---------------+--------+
|         治疗费|10820081|
|           null|    8076|
|       中草药费|  665349|
|       其他费用|  450843|
|    乙类药品20%|     814|
|            B超|  298076|
|乙类诊疗项目30%|      17|
|             CT|   53340|
|         注射费|   92783|
|乙类诊疗项目20%|     296|
|         床位费| 1101955|
|         西药费|10426266|
|       中成药费|  362267|
|         挂号费|     332|
|         材料费| 3351462|
|       常规检查| 4844619|
|乙类诊疗项目40%|       8|
|    乙类药品10%|     216|
|         麻醉费|   90391|
|         输血费|   41347|
|         护理费|  701564|
|         化验费|10408675|
|   自费诊疗项目|    1215|
|         手术费|  203922|
|         放射费|  271087|
|         输氧费|      70|
+---------------+--------+



In [24]:
detail.filter(detail["CHARGE_TYPE"]=="手术费").show(10)

+--------------------+--------------------+-----------------+----------------+---------------------+------------+------------------------+-------------------+-------------------+-----------+---------------------+---------+-----+--------+-----+--------------+------------+-----------+-----------------+
|    CHARGE_DETAIL_NO|         REGISTER_NO|EXTERNAL_CD_STR_1|MED_PROJECT_CODE|     MED_PROJECT_NAME|PROJECT_TYPE|       HOSP_PROJECT_NAME|    SETTLEMENT_DATE|        RECIPE_DATE|CHARGE_TYPE|MEDICAL_SPECIFICATION|DOSE_FORM|PRICE|QUANTITY|MONEY|CD_OVERALL_PAY|APPLY_AMOUNT|SELF_AMOUNT|HOSP_PROJECT_CODE|
+--------------------+--------------------+-----------------+----------------+---------------------+------------+------------------------+-------------------+-------------------+-----------+---------------------+---------+-----+--------+-----+--------------+------------+-----------+-----------------+
|219935141-1-60705...|219935141-1-33052...|        219958141|    f33080201500|     主肺动脉窗修补术|  

### 每个住院样本明细解析

In [17]:
a = detail.filter(detail['REGISTER_NO']=="220019279-1-330522012").toPandas()
a

,CHARGE_DETAIL_NO,REGISTER_NO,EXTERNAL_CD_STR_1,MED_PROJECT_CODE,MED_PROJECT_NAME,PROJECT_TYPE,HOSP_PROJECT_NAME,SETTLEMENT_DATE,RECIPE_DATE,CHARGE_TYPE,MEDICAL_SPECIFICATION,DOSE_FORM,PRICE,QUANTITY,MONEY,CD_OVERALL_PAY,APPLY_AMOUNT,SELF_AMOUNT,HOSP_PROJECT_CODE
0,220019279-1-607757905-104,220019279-1-330522012,220042279,C01100201021056y,甘草,药品费,炙甘草,2019-01-15 00:00:00,2019-01-11 00:00:00,中草药费,统,None,0.12,-1.0,-0.12,-0.12,-0.12,0.0,None
1,220019279-1-607757886-174,220019279-1-330522012,220042279,f41000000100,贴敷疗法,诊疗费,贴敷疗法,2019-01-15 00:00:00,2019-01-12 00:00:00,治疗费,无,None,10.0,1.0,10.0,10.0,10.0,0.0,None
2,220019279-1-607756954-31,220019279-1-330522012,220042279,C03100101011081y,麻黄,药品费,炙麻黄,2019-01-15 00:00:00,2019-01-11 00:00:00,中草药费,1g,None,0.15,-7.0,-1.05,-1.05,-1.05,0.0,None
3,220019279-1-607756962-178,220019279-1-330522012,220042279,f11020000503,住院诊查费（一级医院),诊疗费,住院诊查费(一级医院),2019-01-15 00:00:00,2019-01-13 00:00:00,常规检查,无,None,2.0,1.0,2.0,2.0,2.0,0.0,None
4,220019279-1-607756947-161,220019279-1-330522012,220042279,C02200201011179y,苦杏仁,药品费,苦杏仁,2019-01-15 00:00:00,2019-01-11 00:00:00,中草药费,统,None,0.1356,-9.0,-1.22,-1.16,-1.16,0.0,None
5,220019279-1-607757028-62,220019279-1-330522012,220042279,C01100201021056y,甘草,药品费,炙甘草,2019-01-15 00:00:00,2019-01-11 00:00:00,中草药费,统,None,0.12,30.0,3.6,3.6,3.6,0.0,None
6,220019279-1-607755971-8,220019279-1-330522012,220042279,x140100006681013,氨溴索,药品费,盐酸氨溴索注射液,2019-01-15 00:00:00,2019-01-09 00:00:00,西药费,4ml:30mg*1,注射剂,0.21,90.0,18.9,15.12,15.12,3.7799999999999994,None
7,220019279-1-607757878-108,220019279-1-330522012,220042279,C03100101011142y,鱼腥草,药品费,鱼腥草,2019-01-15 00:00:00,2019-01-11 00:00:00,中草药费,统,None,0.05,100.0,5.0,5.0,5.0,0.0,None
8,220019279-1-607757884-167,220019279-1-330522012,220042279,f12040000601,静脉输液（住院/含输液器）,诊疗费,静脉输液(住院/含输液器),2019-01-15 00:00:00,2019-01-12 00:00:00,治疗费,无,None,9.0,1.0,9.0,9.0,9.0,0.0,None
9,220019279-1-607757934-58,220019279-1-330522012,220042279,C07100101011010y,桂枝,药品费,炙桂枝,2019-01-15 00:00:00,2019-01-11 00:00:00,中草药费,统,None,0.04,-1.0,-0.04,-0.04,-0.04,0.0,None


In [43]:
detail.groupby("CHARGE_TYPE").count().sort("count").show(1000)

+---------------+--------+
|    CHARGE_TYPE|   count|
+---------------+--------+
|乙类诊疗项目40%|       8|
|乙类诊疗项目30%|      17|
|         输氧费|      70|
|    乙类药品10%|     216|
|乙类诊疗项目20%|     296|
|         挂号费|     332|
|    乙类药品20%|     814|
|   自费诊疗项目|    1215|
|           null|    8076|
|         输血费|   41347|
|             CT|   53340|
|         麻醉费|   90391|
|         注射费|   92783|
|         手术费|  203922|
|         放射费|  271087|
|            B超|  298076|
|       中成药费|  362267|
|       其他费用|  450843|
|       中草药费|  665349|
|         护理费|  701564|
|         床位费| 1101955|
|         材料费| 3351462|
|       常规检查| 4844619|
|         化验费|10408675|
|         西药费|10426266|
|         治疗费|10820081|
+---------------+--------+



In [45]:
a[a.CHARGE_TYPE=='治疗费']

,CHARGE_DETAIL_NO,REGISTER_NO,EXTERNAL_CD_STR_1,MED_PROJECT_CODE,MED_PROJECT_NAME,PROJECT_TYPE,HOSP_PROJECT_NAME,SETTLEMENT_DATE,RECIPE_DATE,CHARGE_TYPE,MEDICAL_SPECIFICATION,DOSE_FORM,PRICE,QUANTITY,MONEY,CD_OVERALL_PAY,APPLY_AMOUNT,SELF_AMOUNT,HOSP_PROJECT_CODE
175,220019279-1-607755964-12,220019279-1-330522012,220042279,f21010201501,数字化摄影(DR),诊疗费,数字化摄影(DR),2019-01-15 00:00:00,2019-01-09 00:00:00,放射费,无,None,35.0,2.0,70.0,56.0,56.0,14.0,None


In [69]:
import pandas as pd

In [28]:
a["MONEY"] = pd.to_numeric(a['MONEY'], errors='coerce').fillna(0)
dict(a.groupby('CHARGE_TYPE')['MONEY'].sum())

{'中成药费': 59.61,
 '中草药费': 391.4099999999999,
 '化验费': 54.0,
 '常规检查': 12.0,
 '床位费': 120.0,
 '放射费': 70.0,
 '材料费': 19.36,
 '治疗费': 161.3,
 '西药费': 927.2599999999996}

In [50]:
# from pyspark.sql.types import *

# pyspark2.42 does not support panda_udf well
# from pyspark.sql.functions import pandas_udf,PandasUDFType
# import pandas as pd

# import pdb
# @pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)
# def fee_calc(df):
#     register_no = df['REGISTER_NO'].iloc[0]
#     df["MONEY"] = pd.to_numeric(df['MONEY'], errors='coerce').fillna(0)
#     fee_dict = dict(df.groupby(['CHARGE_TYPE'])['MONEY'].sum())
#     if '手术费' in fee_dict:
#         has_surgery = 1
#         sugery_fee = fee_dict['手术费']
#     else:
#         has_surgery = 0
#         sugery_fee = 0
    
#     herb_fee = fee_dict.get("中草药费", 0)
#     western_fee = fee_dict.get("西药费", 0)
#     cn_med_fee = fee_dict.get("中成药费", 0)
#     bed_fee = fee_dict.get("床位费",0)
#     material_fee = fee_dict.get("材料费",0)
#     care_fee = fee_dict.get("护理费",0)
#     treat_fee = fee_dict.get("治疗费",0)

#     details = ",".join(df.MED_PROJECT_NAME.dropna().unique())
    
#     return pd.DataFrame([[register_no]+[details]+[has_surgery]+[sugery_fee]+
#                          [herb_fee]+[western_fee]+[cn_med_fee]+[bed_fee]+
#                          [material_fee]+[care_fee]+[treat_fee]])

from pyspark.sql.functions import col, collect_list, concat_ws, udf

from pyspark.sql import Row 


schema = StructType([
    StructField("details", StringType()),
    StructField("has_surgery", IntegerType()),
    StructField("sugery_fee", DoubleType()),
    StructField("herb_fee", DoubleType()),
    StructField("western_fee", DoubleType()),
    StructField("cn_med_fee", DoubleType()),
    StructField("bed_fee", DoubleType()),
    StructField("material_fee", DoubleType()),
    StructField("care_fee", DoubleType()),
    StructField("treat_fee", DoubleType()),
])


def myFunc(data_list):
    fee_dict = {}
    details_set = []
    for x in data_list:
        detailname, charge_type, money = x.split(',')
        money_sum = fee_dict.get(charge_type, 0)
        money_sum += float(money)
        fee_dict[charge_type] = money_sum
        details_set += [detailname]
    
    if '手术费' in fee_dict:
        has_surgery = 1
    else:
        has_surgery = 0
    sugery_fee =  fee_dict.get("手术费", 0)
    herb_fee = fee_dict.get("中草药费", 0)
    western_fee = fee_dict.get("西药费", 0)
    cn_med_fee = fee_dict.get("中成药费", 0)
    bed_fee = fee_dict.get("床位费",0)
    material_fee = fee_dict.get("材料费",0)
    care_fee = fee_dict.get("护理费",0)
    treat_fee = fee_dict.get("治疗费",0)
    details = ",".join(list(set(details_set)))

    return Row('details', 'has_surgery',"sugery_fee","herb_fee",
               "western_fee","cn_med_fee","bed_fee","material_fee","care_fee","treat_fee")(details,has_surgery,sugery_fee,herb_fee,western_fee,cn_med_fee,bed_fee,material_fee,care_fee,treat_fee)

myUdf = udf(myFunc, schema)

**combine register and detail**

In [12]:
register= register.filter(register['out_hosp_diagnosisname'].isNotNull())

In [51]:
detail_sumary = detail.na.fill("").withColumn('data', concat_ws(',', col('MED_PROJECT_NAME'), col('CHARGE_TYPE'), col('MONEY'))) \
  .groupBy('REGISTER_NO').agg(collect_list('data').alias('data'))\
  .withColumn('data', myUdf('data'))

detail_sumary = detail_sumary.select("REGISTER_NO", "data.*")

detail_sumary.show(truncate=False)


+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
combined = register.join(detail_sumary, ["register_no"])


In [54]:
combined.show()

+--------------------+----------+---+---+--------+----------+---------------------+--------------------------+--------------------------+----------------------+----------+-------------------+-------------------+------------+----------+----------+----------+-----------+-----------+--------+----------+---------+--------------+-------------------+------------------------------------+-----------+----------+------------------+------------------+------------------+-------+------------------+--------+---------+
|         register_no|    per_no|age|sex|per_type| zone_name|in_hosp_diagnosiscode|     in_hosp_diagnosisname|    out_hosp_diagnosisname|out_hosp_diagnosiscode|visit_type|       in_hosp_date|      out_hosp_date|     hosp_no| hosp_name|hosp_level| dept_name|doctor_name|doctor_code|bill_sum|bill_apply|bill_self|rs_overall_pay|    settlement_date|                             details|has_surgery|sugery_fee|          herb_fee|       western_fee|        cn_med_fee|bed_fee|      material_fee|

In [62]:
# 把丫存起来
combined.coalesce(1).write.parquet(COMBINED_PATH)

# 3.**Catboost 机器学习模型**



## 3.1 特征工程 和 EDA

In [56]:
# pick ICDs with enough sample
import pyspark.sql.functions as Pyspark_F

from pyspark import SparkConf
from pyspark.sql import SparkSession

# conf = SparkConf().setAppName('BZGT').setMaster('local[*]')
# spark = SparkSession.builder.config(conf=conf).config("spark.driver.memory", "35g").config("spark.local.dir", "/media/data1/data/tmp").getOrCreate()
# spark
# #从医保报销的单据表中抽取住院单据以及其对应明细清单
# register = spark.read.parquet(REGISTER_PATH)
# detail = spark.read.parquet(DETAIL_PATH)
# combine = spark.read.parquet(COMBINED_PATH)
# print(register.count())
# print(detail.count())
# print(combine.count())

In [63]:
combine = spark.read.parquet(COMBINED_PATH)
combine.show()

+--------------------+----------+---+---+--------+----------+---------------------+--------------------------+--------------------------+----------------------+----------+-------------------+-------------------+------------+----------+----------+----------+-----------+-----------+--------+----------+---------+--------------+-------------------+------------------------------------+-----------+----------+------------------+------------------+------------------+-------+------------------+--------+---------+
|         register_no|    per_no|age|sex|per_type| zone_name|in_hosp_diagnosiscode|     in_hosp_diagnosisname|    out_hosp_diagnosisname|out_hosp_diagnosiscode|visit_type|       in_hosp_date|      out_hosp_date|     hosp_no| hosp_name|hosp_level| dept_name|doctor_name|doctor_code|bill_sum|bill_apply|bill_self|rs_overall_pay|    settlement_date|                             details|has_surgery|sugery_fee|          herb_fee|       western_fee|        cn_med_fee|bed_fee|      material_fee|

In [65]:
### filter out diseases with few samples

min_sample = 200

icd_diagnosis = \
register.select(['out_hosp_diagnosisname','out_hosp_diagnosiscode']).dropna().\
filter(Pyspark_F.length("out_hosp_diagnosiscode")>=5).\
withColumn("ICD4", register.out_hosp_diagnosiscode.substr(0,5)).\
groupby(['out_hosp_diagnosisname','ICD4']).count().\
filter(f"count >= {min_sample}").\
sort(["out_hosp_diagnosisname",]).collect()

icd_diagnosis_dict = {}
for row in icd_diagnosis:
    icd_diagnosis_dict[row.ICD4] = row.out_hosp_diagnosisname
len(icd_diagnosis_dict)

118

In [67]:
### 查看数据

data = \
combined.filter(Pyspark_F.length("out_hosp_diagnosiscode")>=5).\
withColumn("ICD4", combined.out_hosp_diagnosiscode.substr(0,5)).\
filter(Pyspark_F.col('ICD4').isin(list(icd_diagnosis_dict.keys()))).toPandas()
data

,register_no,per_no,age,sex,per_type,zone_name,in_hosp_diagnosiscode,in_hosp_diagnosisname,out_hosp_diagnosisname,out_hosp_diagnosiscode,...,has_surgery,sugery_fee,herb_fee,western_fee,cn_med_fee,bed_fee,material_fee,care_fee,treat_fee,ICD4
0,219762348-1-330522012,f91c34b573,36,女,城镇职工,c7cdeaff4b,K29.500,慢性胃炎,慢性胃炎,K29.500,...,1,450.00,NaN,224.85,172.00,70.0,122.40,NaN,59.30,K29.5
1,219762549-1-330522012,fe114e1d78,56,男,城镇职工,c7cdeaff4b,M13.900,关节炎,半月板损伤,M23.308,...,1,2607.01,NaN,2426.48,NaN,150.0,1537.99,NaN,1900.33,M23.3
2,219786156-1-330522012,946df375b9,56,男,城镇职工,c7cdeaff4b,R42.x00,头晕和眩晕,头晕和眩晕,R42.x00,...,0,NaN,NaN,264.78,NaN,13.0,47.15,NaN,87.00,R42.x
3,219851040-1-330522012,5a605afc4b,64,男,城镇居民,c7cdeaff4b,N40.x00,前列腺增生,前列腺增生,N40.x00,...,0,NaN,NaN,348.48,NaN,46.0,8.75,NaN,223.30,N40.x
4,219851061-1-330522012,79af746d7c,27,女,城镇居民,c7cdeaff4b,syjtbx,生育津贴报销,生育津贴报销,syjtbx,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syjtb
5,219851488-1-330522012,255f7572c7,76,男,城镇居民,c7cdeaff4b,K80.200-01,胆囊结石,胆囊结石,K80.200-01,...,0,NaN,NaN,1849.39,38.88,100.0,32.00,NaN,91.00,K80.2
6,219908562-1-330522012,f40f42aefa,86,男,城镇居民,c7cdeaff4b,J44.100,慢性阻塞性肺病伴有急性加重,慢性阻塞性肺病伴有急性加重,J44.100,...,1,0.01,885.72,1988.88,NaN,320.0,125.96,NaN,2945.00,J44.1
7,219908631-1-330522012,1df8f0475b,79,男,城镇职工,c7cdeaff4b,R42.x00,头晕和眩晕,头晕和眩晕,R42.x00,...,0,NaN,680.12,1417.21,101.86,180.0,140.40,NaN,739.00,R42.x
8,219935082-1-330522012,ccb2835bd1,72,男,城镇职工,c7cdeaff4b,J44.100,慢性阻塞性肺病伴有急性加重,慢性阻塞性肺病伴有急性加重,J44.100,...,0,NaN,NaN,2245.67,NaN,180.0,101.46,NaN,1485.00,J44.1
9,219986664-1-330522012,61bf28584e,68,男,城镇职工,c7cdeaff4b,I10.x00,特发性(原发性)高血压,特发性(原发性)高血压,I10.x00,...,0,NaN,NaN,706.32,303.96,205.0,76.36,NaN,297.00,I10.x


In [77]:
data.set_index("ICD4", inplace=True)


In [78]:
data.dtypes

register_no                object
per_no                     object
age                         int32
sex                        object
per_type                   object
zone_name                  object
in_hosp_diagnosiscode      object
in_hosp_diagnosisname      object
out_hosp_diagnosisname     object
out_hosp_diagnosiscode     object
visit_type                 object
in_hosp_date               object
out_hosp_date              object
hosp_no                    object
hosp_name                  object
hosp_level                 object
dept_name                  object
doctor_name                object
doctor_code                object
bill_sum                   object
bill_apply                 object
bill_self                  object
rs_overall_pay             object
settlement_date            object
details                    object
has_surgery                 int32
sugery_fee                float64
herb_fee                  float64
western_fee               float64
cn_med_fee    

### calculate inHospital days and clear dataformat


In [ ]:
import pandas as pd
num_cols = ['bill_sum','bill_apply','bill_self','sugery_fee','herb_fee',
            'western_fee','cn_med_fee','bed_fee','material_fee','care_fee','treat_fee']

date_cols = ['settlement_date','in_hosp_date','out_hosp_date']

data[num_cols] = data[num_cols].apply(pd.to_numeric, errors='coerce')
data[date_cols]=data[date_cols].apply(pd.to_datetime, errors='coerce')
data["inpatient_days"] =(data.out_hosp_date - data.in_hosp_date).dt.days
data.set_index("ICD4", inplace=True)
data.dtypes

### 数据可视化，启动不起来就算了

In [75]:
import param
import hvplot.pandas  # noqa
import panel as pn

class DataAnalyzer(param.Parameterized):
    choice = param.Selector(objects=list(icd_diagnosis_dict.keys()))
    
#     has_sugury = param.Boolean()
    select_columns   = param.ListSelector(default=['bill_sum'], objects=num_cols+['inpatient_days'], )
 
    @param.depends('choice', 'select_columns')
    def view(self):
        df = data[(data.index ==self.choice) & (data.has_surgery==1)]
        df2 = data[(data.index ==self.choice) & (data.has_surgery==0)]
        with_surg = df.hvplot.hist(title=f"有手术:样本数目{len(df)}",y=self.select_columns,value_label='费用',alpha=0.5,legend='top_right')
        without_surg = df2.hvplot.hist(title=f"没有手术:样本数目{len(df2)}",y=self.select_columns,value_label='费用',alpha=0.5,legend='top_right')
        col = pn.Column()
        col.append(icd_diagnosis_dict[self.choice])
        col.append((with_surg + without_surg).cols(1))
        return col
da = DataAnalyzer(name='ICD4 fee distribution')

pn.Row(da.param,da.view)

Row
    [0] Column(margin=5, name='ICD4 fee distribution', width=300)
        [0] StaticText(value='<b>ICD4 fee d...)
        [1] Select(name='Choice', options=OrderedDict([('E11.9', ...]), value='E11.9')
        [2] MultiSelect(name='Select columns', options=OrderedDict([('bill_sum', ...]), value=['bill_sum'])
    [1] ParamMethod(method)

### 特征工程

In [76]:
# details 文本特征
# 使用 NMF 主题模型 进行文本向量化
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn import preprocessing

X = data['details']

n_features = 2000
n_components = 50
n_top_words = 100

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [77]:
data.details

ICD4
K29.5    ABO血型鉴定,凝血功能常规检查,乙肝核心抗体测定,血清丙氨酸氨基转移酶测定,丙泊酚,血清碱...
M23.3    ,氨基葡萄糖,丙泊酚,一次性鼻导管,血清碱性磷酸酶测定,普通病房床位费,血浆D-二聚体测定(...
R42.x    雷贝拉唑,丙泊酚,血清碱性磷酸酶测定,红细胞沉降率测定(ESR),普通病房床位费,血浆D-二...
N40.x    ,凝血功能常规检查,庚型肝炎IgM抗体测定,乙肝核心抗体测定,肺通气功能检查,C—反应蛋白测...
syjtb                                                     
K80.2    ,血清碱性磷酸酶测定,血清纤维连接蛋白测定,普通病房床位费,血浆D-二聚体测定(D-Dime...
J44.1    结核杆菌抗体测定,竹茹（配方颗粒）,一次性鼻导管,血清碱性磷酸酶测定,免疫荧光染色诊断,普通...
R42.x    ,颅内段血管彩色多普勒超声,雷贝拉唑,参麦注射液,一次性鼻导管,血清碱性磷酸酶测定,红细胞沉...
J44.1    ,血清碱性磷酸酶测定,普通病房床位费,血浆D-二聚体测定(D-Dimer),血清高密度脂蛋白...
I10.x    ,雷贝拉唑,普通病房床位费,血浆D-二聚体测定(D-Dimer),常规心电图检查,血浆肾素活...
R10.4    ,血清碱性磷酸酶测定,普通病房床位费,血清高密度脂蛋白胆固醇测定,无机磷测定,血清白蛋白测定...
I10.x    ,一次性鼻导管,血清碱性磷酸酶测定,普通病房床位费,血清高密度脂蛋白胆固醇测定,无机磷测定,...
C50.9    尿尿酸测定,神农镇痛膏,血清碱性磷酸酶测定,红细胞沉降率测定(ESR),莪术,普通病房床位费...
L72.1    ABO血型鉴定,活化部分凝血活酶时间测定(APTT),脂肪血管瘤切除术(小),穿刺组织活检检...
N73.9    ,厌氧菌培养及鉴定,计算机图文报告,普通病房床位费,即毁式一次性阴道扩张器,血清人绒毛膜促性...
J06.9    ,厌氧菌培养及鉴定,胎儿心率电子监测,绷带,凝血功能常规检查,糖化血红蛋白测定,胎儿声音刺激...
K56.7    血清碱性磷酸酶测定,普通病房床位费,血浆D-二聚体测定(D-Dimer),血清高密度

In [78]:
print("Extracting tf-idf features for NMF...")
stop_words= ["、",")(","(","（","）",")","+",'-','[',']','、']
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, 
                                   min_df=30,
                                   analyzer="char_wb",ngram_range=(2,4),
                                   max_features=n_features,
                                   stop_words=stop_words)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(X)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 102.704s.


In [79]:
tfidf.shape

(84978, 2000)

In [81]:
#tfidf_vectorizer.build_analyzer()("瑞舒伐他汀,ABO血型鉴定,肝功能常规检查,铁测定,血清载脂蛋白AⅠ测定")

In [91]:
# tfidf_vectorizer.build_tokenizer()("瑞舒伐他汀,ABO血型鉴定,肝功能常规检查,铁测定,")

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (len(X), n_features))
t0 = time()
## 注意！！！ 比赛时候训练全量数据
nmf = NMF(n_components=n_components, 
          random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf[:5000])
print("done in %0.3fs." % (time() - t0))

t0 = time()
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

## 注意！！！ 比赛时候训练全量数据
X_nmf=nmf.transform(tfidf[:5000])
X_nmf.shape
print("done in %0.3fs." % (time() - t0))
# 费用特征
# categorical feature

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=84978 and n_features=2000...


/media/data1/ubuntu/repository/zhaohang/RiskAnalytics/src/main/python/venv/lib/python3.6/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


done in 35.925s.

Topics in NMF model (Frobenius norm):
Topic #0: 管, 尼, ,气管 ,气 甲状腺 状腺 气管 疗, 肽酶测 肽酶测定 氨基肽酶 氨基肽 肽酶 基肽 基肽酶测 基肽酶 手术 二级 （二级医 （二级 二级医 二级医院 ,抗甲状 ,抗甲 抗甲状 抗甲状腺 抗甲 （二 ,甘 监护 护, 监护, ,红 （1 嗪, 中药 ,中药 ,中 管紧 管紧张 血管紧张 血管紧 紧张 表面 治疗, 酰氨基 酰氨 ,血清亮 清亮氨酰 清亮氨 清亮 氨酰氨基 亮氨 血清亮 氨酰氨 亮氨酰 血清亮氨 酰氨基肽 亮氨酰氨 ,肝素 肝素 ,穴位贴 敷治疗, 位贴敷治 位贴 贴敷治疗 穴位贴敷 穴位贴 位贴敷 贴敷治 敷治疗 敷治 ,穴位 ,穴 定,自费 定,自 皮试, 试, ,皮试, 皮试 ,皮试 卡因, 因, 测定,自 素, ,血清游 t3) 状原氨酸 碘甲状原 状原氨 状原 氨酸( 原氨酸 碘甲 碘甲状 原氨 t3)测 原氨酸( 三碘 甲状原
Topic #1: 配方 方颗 配方颗 配方颗粒 方颗粒） （配 （配方 （配方颗 方颗粒 颗粒） 粒） 颗粒）, 粒）, 颗粒 粒）,血 草（ 甘草 ,白 ,甘 阴道 ）,一 ）,静 ）,静脉 ）,抗 ,地 ）,一次 ,大 治疗, 疗, 中药 ,中药 贴敷 （二级医 （二级 二级医院 二级医 血管紧 管紧张 管紧 血管紧张 紧张 二级 （二 na 血管 ）,氯 表面 化酶 化酶测 化酶测定 查费（二 费（二 费（二级 ,中 e测定 e测 e测定, ,穴位贴 ,穴位 ,穴 敷治疗, 白e测 白e测定 蛋白e测 蛋白e 白e 脂蛋白e (抗 敷治 穴位贴 穴位贴敷 位贴 位贴敷治 位贴敷 敷治疗 贴敷治 贴敷治疗 )（ 敷疗法, 敷疗法 贴敷疗 敷疗 贴敷疗法 ,皮 β2 2微球 2微球蛋 β2微球 2微 微球 微球蛋 微球蛋白 β2微 ,贴敷疗 ,贴 ,贴敷 ,β2 ,β2微 ,糖化 ,糖化血
Topic #2: 急诊） 诊） 急诊）, 诊）, 急诊 （急 （急诊 （急诊） 测定（急 定（急诊 定（急 定（ 测定（ 诊）,血 钾测定（ 钠测定（ 氯测定（ 钙测定（ hc (hc 酶测定( ,动 动脉 ）,静 ）,静脉 定,钾 定,钾测 尿素氮 尿素氮测 素氮 氮测 氮测定 

(5000, 50)

In [158]:
# le = preprocessing.LabelEncoder()
# le.fit(train_data_icd.label)

# y = pd.Series(le.transform(train_data_icd.label))
data.dtypes

register_no                       object
per_no                            object
age                                int32
sex                               object
per_type                          object
zone_name                         object
in_hosp_diagnosiscode             object
in_hosp_diagnosisname             object
out_hosp_diagnosisname            object
out_hosp_diagnosiscode            object
visit_type                        object
in_hosp_date              datetime64[ns]
out_hosp_date             datetime64[ns]
hosp_no                           object
hosp_name                         object
hosp_level                        object
dept_name                         object
doctor_name                       object
doctor_code                       object
bill_sum                         float64
bill_apply                       float64
bill_self                        float64
rs_overall_pay                    object
settlement_date           datetime64[ns]
details         

In [94]:
from catboost import Pool, CatBoostClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.cluster import FeatureAgglomeration
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import confusion_matrix

set_config(display='diagram')

In [95]:
categorical_cols = ["sex","zone_name","hosp_no","hosp_level",'has_surgery']
numerical_cols = ["age","bill_sum","sugery_fee","inpatient_days","herb_fee",
                  "western_fee","cn_med_fee","bed_fee",
                  "material_fee","care_fee","treat_fee"]
details_col= ['details']

X = data[categorical_cols+numerical_cols+details_col][:10000]
y = np.array(data.index)
X.shape, y.shape

((10000, 17), (84978,))

In [174]:
X.zone_name.value_counts()

c7cdeaff4b    40956
eacdb2c782    28522
acd9db4c1e    18292
f3ab3890d4    13785
Name: zone_name, dtype: int64

In [97]:
stop_words= ["、",")(","(","（","）",")","+",'-','[',']','、']

n_features=2000
n_components = 50

def detail_pipeline():
    func =  FunctionTransformer(lambda df:df.details, validate=False)

    tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=1,
                                   analyzer="char_wb",ngram_range=(2,4),
                                   max_features=n_features,
                                   stop_words=stop_words)
    nmf = NMF(n_components=n_components, 
          random_state=1,
          alpha=.1, l1_ratio=.5)
    #umap 
    return make_pipeline(func,tfidf_vectorizer,nmf)

In [98]:
# set up classifier

from sklearn.utils.class_weight import compute_class_weight
import numpy as np 

    
classes = np.unique(y)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weights = dict(zip(classes, weights))


params = {
    'iterations': 100,
#     'learning_rate': 0.1,
    'task_type':'GPU',
     "devices":'0:1',
#     'random_seed': 144,
     "eval_metric":'Accuracy',
    'loss_function': 'MultiClass',
     "random_strength":0.1,
     "depth":8,
    "leaf_estimation_method":'Newton',
    'class_weights': class_weights,
}

class BoostClassifier(CatBoostClassifier):

    def fit(self, X, y=None, **fit_params):
        #         print(X.filter(regex=f"{categorical_suffix}$").columns.to_list())

        return super().fit(
            X,
            y=y,
            cat_features=categorical_cols,
            **fit_params
        )

In [99]:
def column_mapper(x):
    return pd.DataFrame(data=x, columns=[f"detail_vec_{i}" for i in range(n_components)] + numerical_cols +categorical_cols )

def column_mapper_transformer( other_cols=[]):
    return FunctionTransformer(column_mapper, validate=False)

pred_pipeline = Pipeline([
    ("preprocessing", Pipeline([
        ("columns",  ColumnTransformer([
            ("detail_feature", detail_pipeline(),details_col),
            ('numeric_feature', StandardScaler(), numerical_cols),
        ], remainder='passthrough')),
        ('column_mapper', column_mapper_transformer())
    ])),
    ("estimator", BoostClassifier(**params, logging_level="Silent"))
])
pred_pipeline

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('columns',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('detail_feature',
                                                                   Pipeline(steps=[('functiontransformer',
                                                                                    FunctionTransformer(func=<function detail_pipeline.<locals>.<lambda> at 0x7fe4024a0620>)),
                                                                                   ('tfidfvectorizer',
                                                                                    TfidfVectorizer(analyzer='char_wb',
                                                                                                    max_df=0.5,
                                                                                                    max_features=2000,
                                                                                                    ngram_r...
                                                                                        random_state=1))]),
                                                                   ['details']),
                                                                  ('numeric_feature',
                                                                   StandardScaler(),
                                                                   ['age',
                                                                    'bill_sum',
                                                                    'sugery_fee',
                                                                    'inpatient_days',
                                                                    'herb_fee',
                                                                    'western_fee',
                                                                    'cn_med_fee',
                                                                    'bed_fee',
                                                                    'material_fee',
                                                                    'care_fee',
                                                                    'treat_fee'])])),
                                 ('column_mapper',
                                  FunctionTransformer(func=<function column_mapper at 0x7fe48c1c4b70>))])),
                ('estimator',
                 <__main__.BoostClassifier object at 0x7fe4526720f0>)])

In [ ]:
# handle imbalanced datasets
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(test_size=0.1, random_state=0)
for train_index, test_index in sss.split(X, y):
    train_X, test_X = X.iloc[train_index], X.iloc[test_index]
    train_y, test_y = y[train_index], y[test_index]

train_X.shape,train_y.shape,test_X.shape,test_y.shape

In [296]:
pred_pipeline.fit(train_X,train_y)

/media/data1/ubuntu/repository/zhaohang/RiskAnalytics/src/main/python/venv/lib/python3.6/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('columns',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('detail_feature',
                                                                   Pipeline(steps=[('functiontransformer',
                                                                                    FunctionTransformer(func=<function detail_pipeline.<locals>.<lambda> at 0x7fc625a4d158>)),
                                                                                   ('tfidfvectorizer',
                                                                                    TfidfVectorizer(analyzer='char_wb',
                                                                                                    max_df=0.5,
                                                                                                    max_features=2000,
                                                                                                    ngram_r...
                                                                                        random_state=1))]),
                                                                   ['details']),
                                                                  ('numeric_feature',
                                                                   StandardScaler(),
                                                                   ['age',
                                                                    'bill_sum',
                                                                    'sugery_fee',
                                                                    'inpatient_days',
                                                                    'herb_fee',
                                                                    'western_fee',
                                                                    'cn_med_fee',
                                                                    'bed_fee',
                                                                    'material_fee',
                                                                    'care_fee',
                                                                    'treat_fee'])])),
                                 ('column_mapper',
                                  FunctionTransformer(func=<function column_mapper at 0x7fc625a4d730>))])),
                ('estimator',
                 <__main__.BoostClassifier object at 0x7fc68ff9c240>)])

In [297]:

print(classification_report(test_y, pred_pipeline.predict(test_X)))


              precision    recall  f1-score   support

       A41.9       0.08      0.27      0.13        26
       B99.x       0.17      0.55      0.26        42
       C15.9       0.04      0.07      0.05        27
       C16.9       0.18      0.04      0.06        82
       C18.9       0.20      0.28      0.23        50
       C20.x       0.25      0.14      0.18        51
       C22.9       0.16      0.32      0.21        25
       C34.9       0.47      0.10      0.16       200
       C50.9       0.46      0.75      0.57       114
       C61.x       0.19      0.40      0.26        25
       C73.x       0.77      0.77      0.77        35
       C85.9       0.36      0.48      0.41        33
       C90.0       0.54      0.59      0.56        37
       D17.9       0.41      0.27      0.33        33
       D24.x       0.61      0.90      0.73        30
       D25.9       0.43      0.53      0.47        34
       E04.1       0.90      0.60      0.72        58
       E11.9       0.68    

In [14]:
import seaborn as sn

sn.heatmap(confusion_matrix(test_y, pred_pipeline.predict(test_X)))

NameError: name 'confusion_matrix' is not defined